In [1]:
from ray.rllib.algorithms.ppo import PPOConfig
from mpe2 import simple_v3
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv

def make_env(env_config=None):
    return simple_v3.parallel_env(max_cycles=25, render_mode=None)

register_env("simple_v3", lambda config: PettingZooEnv(make_env(config)))

# Temporary env instance just to grab one agent's space
tmp_env = PettingZooEnv(make_env())
first_agent = list(tmp_env.observation_space.keys())[0]
obs_space = tmp_env.observation_space[first_agent]
act_space = tmp_env.action_space[first_agent]
tmp_env.close()

config = (
    PPOConfig()
    .environment("simple_v3")
    .training(
        train_batch_size_per_learner=2000,
        lr=0.0004,
    )
    .multi_agent(
        policies={"shared_policy": (None, obs_space, act_space, {})},
        policy_mapping_fn=lambda agent_id, *args, **kwargs: "shared_policy",
    )
)

algo = config.build()
print(algo.train())
algo.save("ppo_model_checkpoint")


/Users/paula/.pyenv/versions/part-sim/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
2025-10-07 15:17:31,312	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-10-07 15:17:31,313	WARNING algorithm_config.py:5045 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/Users/paul

PermissionError: [Errno 1] Operation not permitted

In [11]:
from ray.rllib.algorithms.ppo import PPOConfig
from mpe2 import simple_v3
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from supersuit import flatten_v0

def make_env(env_config=None):
    env = simple_v3.parallel_env(max_cycles=25, render_mode=None)  # parallel API is best for RLlib
    return flatten_v0(env)

# Register custom environment with Rllib
register_env("simple_v3", lambda config: PettingZooEnv(make_env(config))) # we also have to use the Pettingzoo wrapper

# Configure.
config = (
    PPOConfig()
    .environment("simple_v3")
    .training(
        train_batch_size_per_learner=2000, # num of steps per agent
        lr=0.0004,
    )
    .multi_agent(
        policies={"shared_policy": (None, make_env().observation_space, make_env().action_space, {})},
        policy_mapping_fn=lambda agent_id, *args, **kwargs: "shared_policy",
    )
)

# Build the Algorithm.
algo = config.build()

# Train for one iteration, which is 2000 timesteps (1 train batch).
print(algo.train())

/Users/paula/.pyenv/versions/part-sim/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/paula/.pyenv/versions/part-sim/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/paula/.pyenv/versions/part-sim/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: T

IndexError: list index out of range

In [2]:
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig

# Configure.
config = (
    PPOConfig()
    .environment("CartPole-v1")
    .training(
        train_batch_size_per_learner=2000,
        lr=0.0004,
    )
)

# Train through Ray Tune.
results = tune.Tuner(
    "PPO",
    param_space=config,
    # Train for 4000 timesteps (2 iterations).
    run_config=tune.RunConfig(stop={"num_env_steps_sampled_lifetime": 4000}),
).fit()

(PPO pid=34554) 2025-10-07 15:18:23,945	WARNING algorithm_config.py:5045 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(PPO pid=34554) [2025-10-07 15:18:24,132 E 34554 5407408] core_worker.cc:2246: Actor with class name: 'SingleAgentEnvRunner' and ID: 'a63804794c79139412fe3b5201000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are lost, the actor restart will fail. See https://github.com/ray-project/ray/issues/53727 for more details.
(SingleAgentEnvRunner pid=34606) 2025-10-07 15:18:27,201	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. 

In [7]:
import pprint
best_result = results.get_best_result()
hyperparam_mutations = ["clip_param", "lr", "num_epochs", "minibatch_size", "train_batch_size_per_learner"]

print("Best performing trial's final set of hyperparameters:\n")
pprint.pprint(
    {k: v for k, v in best_result.config.items()if k in hyperparam_mutations}
)

print("\nBest performing trial's final reported metrics:\n")

metrics_to_print = [
    "episode_reward_mean",
    "episode_reward_max",
    "episode_reward_min",
    "episode_len_mean",
]
pprint.pprint({k: v for k, v in best_result.metrics.items()}) #if k in metrics_to_print})

Best performing trial's final set of hyperparameters:

{'clip_param': 0.3, 'lr': 0.0004, 'minibatch_size': 128, 'num_epochs': 30}

Best performing trial's final reported metrics:

{'config': {'__stderr_file__': None,
            '__stdout_file__': None,
            '_disable_action_flattening': False,
            '_disable_execution_plan_api': -1,
            '_disable_initialize_loss_from_dummy_batch': False,
            '_disable_preprocessor_api': False,
            '_dont_auto_sync_env_runner_states': False,
            '_enable_rl_module_api': -1,
            '_env_to_module_connector': None,
            '_fake_gpus': False,
            '_is_atari': None,
            '_is_online': True,
            '_learner_class': None,
            '_learner_connector': None,
            '_model_config': {},
            '_module_to_env_connector': None,
            '_per_module_overrides': {},
            '_prior_exploration_config': {'type': 'StochasticSampling'},
            '_rl_module_spec':